In [1]:
from scripts.analytics import Analytics
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

calculadora =  Analytics()

In [2]:
def print_report(dono:str):
    res = calculadora.calcular_patrimonio(dono=dono)
    cdi = calculadora.calcular_cdi_proventos(dono=dono)
    ativos = calculadora.calcular_valorizacao_ativos(dono=dono)
    proventos = calculadora.calcular_ativos_proventos(dono=dono)
    
    dolar_hoje = res['dolar_hoje']

    print(f"""
    -------- {dono.title()} Patrimonio --------
    
    Patrimonio total: R${res['patrimonio_total']:,.2f}

    Moeda Real: R${res['patrimonio_moeda_real']:,.2f}
    Ativos Real: R${res['patrimonio_ativos_real']:,.2f}
    Moeda Dolar: ${res['patrimonio_moeda_dolar']:,.2f} (R${res['patrimonio_moeda_dolar']*dolar_hoje:,.2f})
    Ativos Dolar: ${res['patrimonio_ativos_dolar']:,.2f} (R${res['patrimonio_ativos_dolar']*dolar_hoje:,.2f})

    ---------- Detalhamento ----------
    """)


    col_1 = []
    col_2 = []
    col_3 = []
    col_4 = []
    for key, caixa in cdi.items():
        
        if cdi[key]["valor"] <= 0:
            continue

        rendimento_p = (cdi[key]["rendimento"]/cdi[key]["valor"])*100

        col_1.append(key)
        col_2.append(f'R$ {cdi[key]["valor"]:,.2f}')
        col_3.append(f'% {rendimento_p:,.2f}')
        col_4.append(f'R$ {cdi[key]["rendimento"]:,.2f}')

    df_cdi = pd.DataFrame()
    df_cdi['caixa'] = col_1
    df_cdi['valor'] = col_2
    df_cdi['rendimento %'] = col_3
    df_cdi['rendimento'] = col_4

    print(df_cdi.to_string(index=False))
    print('')

    col_1 = []
    col_2 = []
    col_3 = []
    col_4 = []
    valorizacao_carteira = {'investido': 0, 'valor_atual': 0, 'proventotos': 0}
    for key, ativo in ativos.items():
        prov = 0
        if key in proventos:
            prov = proventos[key]['proventos']

        valor = ativo['price_today'] * ativo['quantidade']
        valorizacao = ativo['valorizacao'] * 100

        if key in ['MMM']:
            valorizacao_carteira['investido'] += ativo['valor_investido'] * dolar_hoje
            valorizacao_carteira['valor_atual'] += valor * dolar_hoje
            valorizacao_carteira['proventotos'] += prov * dolar_hoje
        else:
            valorizacao_carteira['investido'] += ativo['valor_investido']
            valorizacao_carteira['valor_atual'] += valor
            valorizacao_carteira['proventotos'] += prov

        if key in ['MMM']:
            tipo_m = '$'
        else:
            tipo_m = 'R$'

        col_1.append(key)
        col_2.append(f'{tipo_m} {valor:,.2f}')
        col_3.append(f'% {valorizacao:,.2f}')
        col_4.append(f'{tipo_m} {prov:,.2f}')

    df_ativos = pd.DataFrame()
    df_ativos['ativo'] = col_1
    df_ativos['valor'] = col_2
    df_ativos['valorização'] = col_3
    df_ativos['proventos'] = col_4

    print(df_ativos.to_string(index=False))

    ativos_valor_atual_mais_proventos = valorizacao_carteira['valor_atual'] + valorizacao_carteira['proventotos']
    ativos_valorizacao = ((valorizacao_carteira['valor_atual']/valorizacao_carteira['investido'])-1)*100
    ativos_valorizacao_mais_proventos = (((valorizacao_carteira['valor_atual']+valorizacao_carteira['proventotos'])/valorizacao_carteira['investido'])-1)*100
    print(f"""
    
    -------- {dono.title()} Carteira --------

    Investido: R${valorizacao_carteira['investido']:,.2f}

    Valor atual: R${valorizacao_carteira['valor_atual']:,.2f}
    Proventos: R${valorizacao_carteira['proventotos']:,.2f}
    Valor atual + Proventos: R${ativos_valor_atual_mais_proventos:,.2f}

    Valorização: %{ativos_valorizacao:,.2f}
    Valorização + Proventos: %{ativos_valorizacao_mais_proventos:,.2f}

    """)

    


In [4]:
print_report('rodrigo')


    -------- Rodrigo Patrimonio --------
    
    Patrimonio total: R$109,406.81

    Moeda Real: R$45,262.00
    Ativos Real: R$41,364.40
    Moeda Dolar: $2,467.94 (R$12,184.96)
    Ativos Dolar: $2,146.00 (R$10,595.45)

    ---------- Detalhamento ----------
    
        caixa        valor rendimento % rendimento
CDB PLUS Itaú R$ 12,481.00       % 0.00    R$ 0.00
  CDB DI Itaú R$ 32,781.00       % 0.00    R$ 0.00

ativo        valor valorização proventos
TAEE3  R$ 3,501.00    % -11.79 R$ 230.56
BBAS3  R$ 4,750.00     % 30.49 R$ 372.01
ITSA4 R$ 16,438.40      % 4.23 R$ 316.54
MGLU3    R$ 283.00    % -19.37   R$ 0.00
ITUB4  R$ 2,753.00      % 1.51  R$ 53.32
SUZB3  R$ 5,093.00      % 2.27   R$ 0.00
CMIG4  R$ 2,496.00      % 7.12  R$ 46.61
IRBR3  R$ 1,386.00     % 38.60   R$ 7.41
TAEE4  R$ 4,664.00     % -0.26   R$ 0.00
  MMM   $ 2,146.00      % 4.87    $ 0.00

    
    -------- Rodrigo Carteira --------

    Investido: R$49,532.69

    Valor atual: R$51,959.85
    Proventos: R$1,026.4

In [5]:
print_report('dayane')


    -------- Dayane Patrimonio --------
    
    Patrimonio total: R$16,652.12

    Moeda Real: R$11,855.38
    Ativos Real: R$0.00
    Moeda Dolar: $435.03 (R$2,147.87)
    Ativos Dolar: $536.50 (R$2,648.86)

    ---------- Detalhamento ----------
    
        caixa        valor rendimento % rendimento
robank_dayane R$ 11,855.38       % 2.32  R$ 275.31

ativo    valor valorização proventos
  MMM $ 536.50      % 4.87    $ 0.00

    
    -------- Dayane Carteira --------

    Investido: R$2,525.92

    Valor atual: R$2,648.86
    Proventos: R$0.00
    Valor atual + Proventos: R$2,648.86

    Valorização: %4.87
    Valorização + Proventos: %4.87

    
